In [30]:
import pickle

In [31]:
import face_recognition
import pickle
import cv2
import base64
import os
import io
from flask import Flask,request,jsonify
import numpy as np
from PIL import Image

In [32]:
data = pickle.loads(open('face_enc (1)', "rb").read())

In [33]:
data['encodings'][0]

array([-1.40422031e-01,  1.41348988e-01,  4.67264913e-02, -2.43981890e-02,
       -7.39150941e-02,  2.76790801e-02, -1.03595912e-01, -4.18067910e-02,
        1.69063315e-01, -1.21135190e-01,  1.18145213e-01, -9.00477543e-03,
       -1.88783675e-01, -6.83860183e-02, -3.49095091e-03,  8.72873887e-02,
       -1.41506284e-01, -1.77093625e-01, -1.27987847e-01, -8.39177743e-02,
       -4.87026647e-02,  3.48472819e-02,  3.27109918e-03,  1.65909082e-02,
       -1.89315468e-01, -3.91346246e-01, -3.11892256e-02, -1.70697689e-01,
        5.64899296e-04, -1.08951047e-01, -4.32581455e-02, -6.26988197e-03,
       -1.32605582e-01, -2.76005119e-02,  1.09529868e-03,  9.98682603e-02,
       -4.71470878e-02, -2.74711195e-02,  2.14020342e-01, -1.30185336e-02,
       -1.74861908e-01, -2.90509891e-02,  8.58195722e-02,  2.54015923e-01,
        2.50192344e-01,  1.12074465e-01,  1.48778278e-02, -2.11099684e-02,
        1.58510402e-01, -2.44570896e-01,  1.12004444e-01,  1.41313478e-01,
        8.84749144e-02,  

In [34]:
image = base64.b64decode(str(file))
image = Image.open(io.BytesIO(image))
image= np.array(image)

NameError: name 'file' is not defined

In [35]:
def predict_rana(image):
    print('Shape of image ',image.shape)
    cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    faceCascade = cv2.CascadeClassifier(cascPathface)
    data = pickle.loads(open('face_enc (1)', "rb").read())
    data1 = pickle.loads(open('user_data (1)', "rb").read())

    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)

    encodings = face_recognition.face_encodings(rgb)
    names = []

    print(len(data["encodings"][0]))

    for encoding in encodings:
        print("in detection")
        print('Shape of encoding = ',encoding.shape)
        print('shape of data = ',len(data['encodings']))
        print(type(encoding),type(data['encodings']))
        matches = face_recognition.compare_faces(data["encodings"],
                                                 encoding)

        print(matches[0]) 

        name = "Unknown"
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
                name = max(counts, key=counts.get)

            names.append(name)
            """for ((x, y, w, h), name) in zip(faces, names):
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                            0.75, (0, 255, 0), 2)"""

    fin_name=max(set(names),key=names.count)
    index=data1["name"].index(fin_name)
    place=data1["place"][index]
    branch=data1["branch"][index]
    year=data1['year'][index]
    studying=data1['studying'][index]
    img = data1["image"][index]
    rawBytes = io.BytesIO()
    img.save(rawBytes, "JPEG")
    rawBytes.seek(0)
    img_base64 = base64.b64encode(rawBytes.read())
    result={'name':fin_name,'place':place,'branch':branch,'image':str(img_base64),'year':year}

    return result

In [36]:
predict_rana(img)

Shape of image  (990, 999, 3)
128
in detection
Shape of encoding =  (128,)
shape of data =  584
<class 'numpy.ndarray'> <class 'list'>
False


{'name': 'Abdullah',
 'place': 'Ranchi Jharkhand',
 'branch': 'Computer Science and Engineering',
 'image': "b'...'",
 'year': '2'}

In [6]:
img=cv2.imread(r"C:\Users\acer\OneDrive\Desktop\Face-of-Campus\Images\Prakhar Pratap Mall\2105030.jpg")

In [149]:
def reset():
    dat={'encodings':[],'names':[]}

    for fol in list(os.listdir('Images/')):
        pat='Images/'+fol+'/'
        for image in list(os.listdir(pat)):
            img_pat=pat+image
            print(img_pat)
            img=cv2.imread(img_pat)
            print(img.shape)
            rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
            faceCascade = cv2.CascadeClassifier(cascPathface)
            faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
            print("Face coordinates = ",faces)
            try:
                for (x, y, w, h) in faces:
                    cv2.rectangle(rgb, (x, y), (x+w, y+h), 
                                (0, 0, 255), 2)
                    faces = rgb[y:y + h, x:x + w]
                print("shape of face = ",faces.shape)       
                cv2.imwrite('face.jpg', faces)
                encodings = face_recognition.face_encodings(faces)
                if(len(encodings)==1):
                    dat['encodings'].append(encodings)
                    dat['names'].append(fol)
                else:
                    continue
            except:
                print("no face found")
    fi = open("face_enc (1)", "wb")
    fi.write(pickle.dumps(dat))
    fi.close()

In [150]:
reset()

Images/Ansh Tandon/1726480e-2dfa-44e3-a3b2-d769637147f3.jpg
(867, 1156, 3)
Face coordinates =  [[577 420  88  88]]
shape of face =  (88, 88, 3)
Images/Ansh Tandon/7f7342e8-d9cb-4bd4-917b-6fd2b74a4861.jpg
(1156, 867, 3)
Face coordinates =  ()
no face found
Images/Ansh Tandon/96b1b113-8cf7-43ba-a0b0-73d928af718a.jpg
(1156, 867, 3)
Face coordinates =  [[302 326 135 135]]
shape of face =  (135, 135, 3)
Images/Ansh Tandon/acba6ffd-2b90-410a-bcf4-80889fa9646a.jpg
(1600, 1200, 3)
Face coordinates =  ()
no face found
Images/Ansh Tandon/ansh1.jpg
(1245, 812, 3)
Face coordinates =  [[249 227 278 278]]
shape of face =  (278, 278, 3)
Images/Ansh Tandon/e68958f8-f0f8-47cc-9f75-25c615a1cf57.jpg
(1280, 720, 3)
Face coordinates =  [[242 241 279 279]]
shape of face =  (279, 279, 3)
Images/Saksham Dwivedi/IMG-20220217-WA0030.jpg
(1280, 583, 3)
Face coordinates =  ()
no face found
Images/Saksham Dwivedi/IMG_20220216_225728~2.jpg
(2196, 1824, 3)
Face coordinates =  [[673 602 376 376]]
shape of face =  (37

In [169]:
data = pickle.loads(open('face_enc (1)', "rb").read())

In [170]:
len(data['encodings'])

14

In [2]:
from scipy.misc import face
import face_recognition
import pickle
import cv2
import base64
import os
import io
import numpy as np
from PIL import Image

In [151]:
def update(image,dict_npb):
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    faceCascade = cv2.CascadeClassifier(cascPathface)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    try:
        for (x, y, w, h) in faces:
            cv2.rectangle(rgb, (x, y), (x+w, y+h), 
                        (0, 0, 255), 2)
            faces = rgb[y:y + h, x:x + w]       
        cv2.imwrite('face.jpg', faces)             
        encodings = face_recognition.face_encodings(faces)
        if(len(encodings)>1):
            print('More than one face detected need single face')
        else:
            list_names=os.listdir('Images/')
            data = pickle.loads(open('face_enc (1)', "rb").read())
            if dict_npb['name'] in list_names:
                cv2.imwrite('Images/'+dict_npb['name']+'/'+str(np.random.randint(1,10000000))+'.jpg',image)
                data['encodings'].append(encodings)
                data['names'].append(dict_npb['name'])
            else:
                os.mkdir('Images/'+dict_npb['name'])
                cv2.imwrite('Images/'+dict_npb['name']+'/'+str(np.random.randint(1,10000000))+'.jpg',image)
                data1 = pickle.loads(open('usernpb', "rb").read())
                data['encodings'].append(encodings)
                data['names'].append(dict_npb['name'])
                data1['name'].append(dict_npb['name'])
                data1['place'].append(dict_npb['place'])
                data1['branch'].append(dict_npb['branch'])
                data1['year'].append(dict_npb['year'])
                data1['studying'].append(dict_npb['studying'])
                fi = open("usernpb", "wb")
                fi.write(pickle.dumps(data1))
                fi.close()
            
            fi = open("face_enc (1)", "wb")
            fi.write(pickle.dumps(data))
            fi.close()
            print('Updated data')
    except:
        print("Face not detected")

In [152]:
dict_np={'name':'Prakhar Pratap Mall','place':'Kanpur','branch':'EE','year':'2nd','studying':'GP'}
img=cv2.imread('prakhar.jpg')

In [154]:
update(img,dict_np)

Updated data


In [174]:
def predict(image):
    cascPathface = os.path.dirname(cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
    faceCascade = cv2.CascadeClassifier(cascPathface)
    data = pickle.loads(open('face_enc (1)', "rb").read())
    data1 = pickle.loads(open('usernpb', "rb").read())

    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)


    try:
        for (x, y, w, h) in faces:
            cv2.rectangle(rgb, (x, y), (x+w, y+h), 
                        (0, 0, 255), 2)
            faces = rgb[y:y + h, x:x + w]       
        cv2.imwrite('face.jpg', faces)   

        encodings = face_recognition.face_encodings(faces)
        names = []

        print('Number of encodings',len(encodings))

        for encoding in encodings:
            matches = face_recognition.compare_faces(data["encodings"],
                                                    encoding)
            
            print(len(matches))
            print(len(matches[0]))
            
            
            name = "Unknown"
            if True in matches:
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1
                    name = max(counts, key=counts.get)

                names.append(name)
                """for ((x, y, w, h), name) in zip(faces, names):
                    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                                0.75, (0, 255, 0), 2)"""

        fin_name=max(set(names),key=names.count)
        index=data1["name"].index(fin_name)
        place=data1["place"][index]
        branch=data1["branch"][index]
        year=data1['year'][index]
        studying=data1['studying'][index]
        imgs=list(os.listdir('Images/'+index))


        # img = data1["image"][index]

        img=cv2.imread('Images/'+index+'/'+imgs[0])
        rawBytes = io.BytesIO()
        img.save(rawBytes, "JPEG")
        rawBytes.seek(0)
        img_base64 = base64.b64encode(rawBytes.read())
        result={'name':fin_name,'place':place,'branch':branch,'image':str(img_base64),'year':year}

    except:
        print("Face not detected")
        result={'name':'','place':'','branch':'','image':'','year':''}
        
        return 

    return result

In [175]:
img=cv2.imread(r'prakhar.jpg')

In [176]:
res=predict(img)

Number of encodings 1
14
128
Face not detected


: 

In [60]:
data1 = pickle.loads(open('usernpb', "rb").read())

In [64]:
data1['place'].append('fsgfg')

In [83]:
img=cv2.imread(r'Images\Tarun Srivastava\1e268cf8-63ca-4171-ad3c-c0253f635a3b.jpg')

In [55]:
cv2.imwrite('Images/'+'Prakhar Pratap Mall'+'/'+str(np.random.randint(1,10000000))+'.jpg',img)

True

In [52]:
print('Images/'+'Prakhar Pratap Mall'+'/'+str(np.random.randint(1,10000000)))

Images/Prakhar Pratap Mall/820341


In [21]:
print(img)

[[[245 243 242]
  [245 243 242]
  [245 243 242]
  ...
  [227 227 233]
  [227 227 233]
  [227 227 233]]

 [[245 243 242]
  [245 243 242]
  [245 243 242]
  ...
  [227 227 233]
  [227 227 233]
  [227 227 233]]

 [[245 243 242]
  [245 243 242]
  [245 243 242]
  ...
  [227 228 232]
  [227 228 232]
  [227 228 232]]

 ...

 [[137 147 157]
  [153 163 173]
  [150 160 170]
  ...
  [ 41  62  47]
  [ 52  71  56]
  [ 57  76  61]]

 [[123 134 142]
  [145 156 164]
  [144 155 163]
  ...
  [ 38  61  46]
  [ 51  70  55]
  [ 56  75  60]]

 [[109 120 128]
  [138 149 157]
  [141 152 160]
  ...
  [ 36  59  44]
  [ 50  69  54]
  [ 56  75  60]]]


In [3]:
data = pickle.loads(open('face_enc (1)', "rb").read())
data1 = pickle.loads(open('user_data (1)', "rb").read())


In [6]:
len(data['encodings'])

584

In [21]:
data.keys()

dict_keys(['encodings', 'names'])

In [9]:
len(data['names'])

584

In [15]:
data1['studying']


['Machine Learning',
 'Android Development',
 'Competitive Programming',
 'Competitive Programming',
 'Vlogging',
 'Electronics Core']

In [16]:
dict_npb={'name':['Saksham Dwivedi','Ansh Tandon','Prakhar Pratap Mall','Tarun Srivastava','Akash Gupta'],'place':['Gorakhpur Uttar Pradesh','Lakhimpurkhiri Uttar Pradesh','Kanpur ','Agra','France'],'branch':['PE','EE','EE','EE','ECE'],'year':[2,2,2,2,2],'studying':['Machine Learning','Android Development','Competitive Programming','Competitive Programming','Vlogging']}

In [17]:
fi = open("usernpb", "wb")
fi.write(pickle.dumps(dict_npb))
fi.close()

In [20]:
dict_npb = pickle.loads(open('usernpb', "rb").read())

In [19]:
dict_npb.keys()

dict_keys(['name', 'place', 'branch', 'year', 'studying'])